In [ ]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install kaggle
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
import numpy as np
from google.colab import drive
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
!pip install optuna
import optuna
import pandas as pd



In [ ]:
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# code for importing and processing the imported data directly from Kaggle
drive.mount('/content/drive')
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/kaggle"
!kaggle competitions download -c vub-ml-project-2024-animal-classification
!mkdir -p /content/drive/MyDrive/kaggle
!unzip vub-ml-project-2024-animal-classification.zip -d /content/

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [ ]:
def ResNet18():
    return ResNet(ResidualBlock)

In [ ]:
# Data preparation and preprocessing
EPOCH = 15
pre_epoch = 0
BATCH_SIZE = 64
LR = 0.01

# Define dataset directory
train_dir = "/content/vub-ml-project-2024-animal-classification/train"

# Define transformations
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),      # switch to 224x224 and fix the network so it will work
    transforms.RandomCrop(224, padding=16),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),

])

# Load the dataset
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)

# Split dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Get the number of classes dynamically
num_classes = len(train_dataset.dataset.classes)

In [ ]:
# no optimization code
# Set hyperparameters
EPOCH = 10
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01

# Define dataset directory
train_dir = "/content/vub-ml-project-2024-animal-classification/train"

# Define transformations
transform_train = transforms.Compose([
    transforms.Resize((32, 32)),  
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),

])

# Load the dataset
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)

# Split dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Get the number of classes dynamically
num_classes = len(train_dataset.dataset.classes)

# Define ResNet18 with the correct number of output classes
net = ResNet(ResidualBlock, num_classes=num_classes).to(device)

# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=10, gamma=0.2)


# StepLR/ExponentialLR: Simple decays for stable training.
# CosineAnnealing: For tasks that benefit from smooth decay.
# ReduceLROnPlateau: Tasks with validation monitoring needs.
# OneCycleLR: Optimized training, often for large-scale tasks.
# CyclicLR: Tasks requiring escape from local minima.

# Training and validation loop
for epoch in range(pre_epoch, EPOCH):
    print(f'\nEpoch: {epoch + 1}')
    net.train()
    sum_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward and backward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Metrics for training
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()

        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%%' %
              (epoch + 1, i + 1, sum_loss / (i + 1), 100. * correct / total))

    # Validation loop
    print('Validating...')
    net.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += predicted.eq(labels.data).cpu().sum()

    print('Validation Loss: %.3f | Validation Acc: %.3f%%' %
          (val_loss / len(val_loader), 100. * correct / total))
scheduler.step()
print(f'Training complete. Total epochs: {EPOCH}')

In [ ]:
# TESTING THE FUNCTIONALITY OF THE CLASS
net = ResNet(ResidualBlock, num_classes=num_classes).to(device)
dummy_input = torch.randn(1, 3, 224, 224).to(device)
output = net(dummy_input)
print("Output shape:", output.shape)

In [ ]:
# no optimization code
# Define ResNet18 with the correct number of output classes
net = ResNet(ResidualBlock, num_classes=num_classes).to(device)

# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
#optimizer = optim.AdamW(net.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=3, gamma=0.4)
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_loader),epochs=EPOCH)



# Training and validation loop
for epoch in range(pre_epoch, EPOCH):
    print(f'\nEpoch: {epoch + 1}')
    net.train()
    sum_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward and backward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Metrics for training
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()

        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%%' %
              (epoch + 1, i + 1, sum_loss / (i + 1), 100. * correct / total))

    # Validation loop
    print('Validating...')
    net.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += predicted.eq(labels.data).cpu().sum()

    print('Validation Loss: %.3f | Validation Acc: %.3f%%' %
          (val_loss / len(val_loader), 100. * correct / total))
    scheduler.step(val_loss / len(val_loader))

#scheduler.step()
print(f'Training complete. Total epochs: {EPOCH}')

In [ ]:
import os
from PIL import Image
from torchvision import transforms

# Define test directory
test_dir = "/content/vub-ml-project-2024-animal-classification/test"

# Define transformations
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Load test images and filenames
test_images = []
image_filenames = []

for filename in os.listdir(test_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(test_dir, filename)
        image = Image.open(image_path).convert("RGB")
        image = transform_test(image)
        test_images.append(image)
        image_filenames.append(filename)

# Stack images into a single tensor
test_images = torch.stack(test_images)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Create a DataLoader from the test images
test_dataset = TensorDataset(test_images)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Define test directory
test_dir = "/content/vub-ml-project-2024-animal-classification/test"

# Define transformations
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Load test images and filenames
test_images = []
image_filenames = []

for filename in os.listdir(test_dir):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        image_path = os.path.join(test_dir, filename)
        image = Image.open(image_path).convert("RGB")
        image = transform_test(image)
        test_images.append(image)
        image_filenames.append(filename)

# Stack images into a single tensor
test_images = torch.stack(test_images)

# Create a DataLoader from the test images
test_dataset = TensorDataset(test_images)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

import csv
import torch.nn.functional as F

# Switch model to evaluation mode
net.eval()

# List to store predictions
test_pred_probs = []

# No need to compute gradients during inference
with torch.no_grad():
    for images in test_loader:
        images = images[0].to(device)  # Use images[0] to unpack the DataLoader tuple

        # Get raw outputs (logits) from the model
        outputs = net(images)

        # Convert logits to probabilities using softmax
        probabilities = F.softmax(outputs, dim=1)

        # Store the probabilities
        test_pred_probs.extend(probabilities.cpu().numpy())

# Get the class labels
label_strings = train_dataset.dataset.classes

# Define the output path for the CSV file
output_path = "./submission/preds.csv"

# Write the predictions to the CSV file
def writePredictionsToCsv(predictions, out_path, label_strings):
    """Writes the predictions to a csv file.
    Assumes the predictions are ordered by test interval id."""
    with open(out_path, 'w') as outfile:
        csvwriter = csv.writer(
            outfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL
        )
        # Write the header
        row_to_write = ['Id'] + [label for label in label_strings]
        csvwriter.writerow(row_to_write)
        # Write the rows using 18 digit precision
        for idx, prediction in enumerate(predictions):
            assert len(prediction) == len(label_strings)
            csvwriter.writerow(
                [idx + 1] +  # Use an integer for the Id
                ["%.18f" % p for p in prediction]
            )

# Call the function to write the predictions to the CSV file
writePredictionsToCsv(test_pred_probs, output_path, label_strings)

print(f"Submission file saved to: {output_path}")
